In [2]:
!pip install pyspark==3.3.1

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession\
    .builder\
    .appName('twitter_transformation')\
    .getOrCreate()

23/05/05 16:52:49 WARN Utils: Your hostname, kirsch-Virtual-Machine resolves to a loopback address: 127.0.1.1; using 172.28.82.115 instead (on interface eth0)
23/05/05 16:52:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/05 16:52:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
df = spark.read.json('../../datalake/twitter_datascience')

In [7]:
df.show()

+--------------------+--------------------+------------------+------------+
|                data|            includes|              meta|extract_date|
+--------------------+--------------------+------------------+------------+
|[{5, 66, 2023-04-...|{[{2023-04-29T21:...|{1234567890abcdef}|  2023-04-29|
|[{19, 21, 2023-04...|{[{2023-04-29T13:...|{1234567890abcdef}|  2023-04-29|
|[{45, 74, 2023-04...|{[{2023-04-29T12:...|{1234567890abcdef}|  2023-04-29|
|[{39, 87, 2023-04...|{[{2023-04-29T11:...|              null|  2023-04-29|
|[{92, 41, 2023-05...|{[{2023-05-03T06:...|{1234567890abcdef}|  2023-05-03|
|[{18, 47, 2023-05...|{[{2023-05-03T20:...|{1234567890abcdef}|  2023-05-03|
|[{54, 80, 2023-05...|{[{2023-05-03T04:...|              null|  2023-05-03|
|[{25, 16, 2023-04...|{[{2023-04-28T18:...|{1234567890abcdef}|  2023-04-28|
|[{9, 17, 2023-04-...|{[{2023-04-28T23:...|              null|  2023-04-28|
|[{20, 100, 2023-0...|{[{2023-05-01T10:...|{1234567890abcdef}|  2023-05-01|
|[{63, 1, 20

In [8]:
df.printSchema()

root
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- author_id: string (nullable = true)
 |    |    |-- conversation_id: string (nullable = true)
 |    |    |-- created_at: string (nullable = true)
 |    |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |    |-- element: long (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- in_reply_to_user_id: string (nullable = true)
 |    |    |-- lang: string (nullable = true)
 |    |    |-- public_metrics: struct (nullable = true)
 |    |    |    |-- like_count: long (nullable = true)
 |    |    |    |-- quote_count: long (nullable = true)
 |    |    |    |-- reply_count: long (nullable = true)
 |    |    |    |-- retweet_count: long (nullable = true)
 |    |    |-- text: string (nullable = true)
 |-- includes: struct (nullable = true)
 |    |-- users: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- c

In [9]:
from pyspark.sql import functions as f

In [10]:
df.select(f.explode('data')).printSchema()

root
 |-- col: struct (nullable = true)
 |    |-- author_id: string (nullable = true)
 |    |-- conversation_id: string (nullable = true)
 |    |-- created_at: string (nullable = true)
 |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- id: string (nullable = true)
 |    |-- in_reply_to_user_id: string (nullable = true)
 |    |-- lang: string (nullable = true)
 |    |-- public_metrics: struct (nullable = true)
 |    |    |-- like_count: long (nullable = true)
 |    |    |-- quote_count: long (nullable = true)
 |    |    |-- reply_count: long (nullable = true)
 |    |    |-- retweet_count: long (nullable = true)
 |    |-- text: string (nullable = true)



In [11]:
df.select(f.explode('data')).show()

+--------------------+
|                 col|
+--------------------+
|{5, 66, 2023-04-2...|
|{22, 45, 2023-04-...|
|{29, 16, 2023-04-...|
|{90, 52, 2023-04-...|
|{97, 0, 2023-04-2...|
|{95, 38, 2023-04-...|
|{64, 27, 2023-04-...|
|{91, 87, 2023-04-...|
|{8, 51, 2023-04-2...|
|{62, 13, 2023-04-...|
|{19, 21, 2023-04-...|
|{12, 86, 2023-04-...|
|{12, 0, 2023-04-2...|
|{29, 20, 2023-04-...|
|{7, 96, 2023-04-2...|
|{49, 95, 2023-04-...|
|{32, 42, 2023-04-...|
|{96, 67, 2023-04-...|
|{87, 98, 2023-04-...|
|{79, 90, 2023-04-...|
+--------------------+
only showing top 20 rows



In [12]:
df.select(f.explode('data').alias('tweets'))\
  .select('tweets.author_id', 'tweets.conversation_id',
          'tweets.created_at', 'tweets.id', 
          'tweets.public_metrics.*', 'tweets.text').printSchema()

root
 |-- author_id: string (nullable = true)
 |-- conversation_id: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- like_count: long (nullable = true)
 |-- quote_count: long (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- text: string (nullable = true)



In [14]:
tweet_df = df.select(f.explode('data').alias('tweets'))\
             .select('tweets.author_id', 'tweets.conversation_id',
                     'tweets.created_at', 'tweets.id', 
                     'tweets.public_metrics.*', 'tweets.text')

tweet_df.show(5)

+---------+---------------+--------------------+---+----------+-----------+-----------+-------------+--------------------+
|author_id|conversation_id|          created_at| id|like_count|quote_count|reply_count|retweet_count|                text|
+---------+---------------+--------------------+---+----------+-----------+-----------+-------------+--------------------+
|        5|             66|2023-04-29T01:57:41Z| 46|        65|          6|         93|           53|Um terceiro tweet...|
|       22|             45|2023-04-29T23:55:35Z| 29|         6|          0|         24|           87|Este é um tweet f...|
|       29|             16|2023-04-29T05:37:04Z| 42|         8|         45|         74|           83|Tweet fictício ge...|
|       90|             52|2023-04-29T17:32:10Z| 49|        50|         93|         51|           24|Tweet fictício ge...|
|       97|              0|2023-04-29T21:32:27Z| 26|        68|         27|         31|           86|Tweet fictício cr...|
+---------+-----

In [18]:
user_df = df.select(f.explode('includes.users').alias('users'))\
             .select('users.created_at', 'users.id',
                     'users.name', 'users.username')

user_df.show(5)

+--------------------+---+------+--------+
|          created_at| id|  name|username|
+--------------------+---+------+--------+
|2023-04-29T21:13:11Z| 16|User 1|   user1|
|2023-04-29T11:21:08Z| 80|User 2|   user2|
|2023-04-29T18:28:47Z| 99|User 3|   user3|
|2023-04-29T10:06:15Z| 51|User 4|   user4|
|2023-04-29T00:27:26Z| 55|User 5|   user5|
+--------------------+---+------+--------+
only showing top 5 rows



In [19]:
tweet_df.coalesce(1).write.mode('overwrite').json('output/tweet')
user_df.coalesce(1).write.mode('overwrite').json('output/user')